## Using ON DEMAND SPARK without HDFS Delegation Token

First run a Spark Job on EMR in cluster mode via `spark-submit` which filters a large dataset to a smaller dataset


In [ ]:
import os
hdfs_endpoint=os.environ['HDFS_ENDPOINT']
!/mnt/code/scripts/my_hdfs.sh dfs -rmr /user/dominospark/smalldata-1000/
!/mnt/code/scripts/filter-data-emr.sh \
1000 hdfs://10.0.123.114:8020 /user/dominospark/largedata/ /user/dominospark/smalldata-1000/

Next copy the smaller dataset created in step 1 in the HDFS into a local one in the project Dataset

**Any output produced by the programs later will need to be copied back to HDFS if needed beyond the lifecycle of the project or for subsequent operations on EMR**

In [ ]:
!rm -rf /mnt/data/ON-DEMAND-SPARK/smalldata-1000
!mkdir /mnt/data/ON-DEMAND-SPARK/smalldata-1000
!/mnt/code/scripts/my_hdfs.sh dfs -copyToLocal /user/dominospark/smalldata-1000/* /mnt/data/ON-DEMAND-SPARK/smalldata-1000/ 

In [ ]:
%env PYSPARK_PYTHON /usr/bin/python3
%env PYSPARK_DRIVER_PYTHON /usr/bin/python3
#%env SPARK_DIST_CLASSPATH  /etc/hadoop/conf:/opt/hadoop/share/hadoop/common/lib/*:/opt/hadoop/share/hadoop/common/*:/usr/lib/hadoop-lzo/lib/hadoop-lzo-0.4.19.jar:/usr/lib/hadoop-lzo/lib/hadoop-lzo.jar:/usr/lib/hadoop-lzo/lib/native:/usr/share/aws/aws-java-sdk/aws-java-sdk-bundle-1.11.977.jar:/usr/share/aws/aws-java-sdk/LICENSE.txt:/usr/share/aws/aws-java-sdk/NOTICE.txt:/usr/share/aws/aws-java-sdk/README.md:/usr/share/aws/emr/emrfs/conf:/usr/share/aws/emr/emrfs/lib/annotations-16.0.2.jar:/usr/share/aws/emr/emrfs/lib/aopalliance-1.0.jar:/usr/share/aws/emr/emrfs/lib/aws-glue-sdk-1.12.0.jar:/usr/share/aws/emr/emrfs/lib/bcprov-ext-jdk15on-1.66.jar:/usr/share/aws/emr/emrfs/lib/emrfs-hadoop-assembly-2.46.0.jar:/usr/share/aws/emr/emrfs/lib/fluent-hc-4.5.6.jar:/usr/share/aws/emr/emrfs/lib/ion-java-1.0.2.jar:/usr/share/aws/emr/emrfs/lib/javax.inject-1.jar:/usr/share/aws/emr/emrfs/lib/jcl-over-slf4j-1.7.21.jar:/usr/share/aws/emr/emrfs/lib/jmespath-java-1.11.852.jar:/usr/share/aws/emr/emrfs/lib/lombok-1.18.4.jar:/usr/share/aws/emr/emrfs/lib/mockito-core-1.10.19.jar:/usr/share/aws/emr/emrfs/lib/objenesis-2.1.jar:/usr/share/aws/emr/emrfs/lib/secret-agent-interface-1.4.0.jar:/usr/share/aws/emr/emrfs/lib/slf4j-api-1.7.21.jar:/usr/share/aws/emr/emrfs/auxlib/*:/usr/share/aws/emr/ddb/lib/emr-ddb-hadoop.jar:/usr/share/aws/emr/goodies/lib/emr-hadoop-goodies.jar:/usr/share/aws/emr/kinesis/lib/emr-kinesis-hadoop.jar:/usr/share/aws/emr/cloudwatch-sink/lib/cloudwatch-sink-2.1.0.jar:/usr/share/aws/emr/cloudwatch-sink/lib/cloudwatch-sink.jar:/opt/hadoop/share/hadoop/tools/lib/*:/usr/lib/hadoop-lzo/lib/*
#%env JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, IntegerType
import random

In [ ]:
sparkSession = SparkSession.builder.appName("ML On Smaller Datasets Copied from HDFS") \
.config("spark.dynamicAllocation.enabled", "false") \
.config("spark.driver.extraClassPath", "/opt/hadoop/etc/hadoop:/usr/lib/hadoop-lzo/lib/*:/opt/hadoop/share/hadoop/mapreduce/*:/opt/hadoop/share/hadoop/hdfs/*") \
.getOrCreate()
sc=sparkSession.sparkContext
#.config("spark.executor.extraClassPath", "/opt/hadoop/etc/hadoop:/usr/lib/hadoop-lzo/lib/*:/opt/hadoop/share/hadoop/mapreduce/*:/opt/hadoop/share/hadoop/hdfs/*") \


In [ ]:
columns = StructType([ StructField("id", IntegerType(), True),
                       StructField("v1", IntegerType(), True),
                       StructField("v2", IntegerType(), True),
                       StructField("v3", IntegerType(), True) ])
df_load = sparkSession.read.csv('file:///mnt/data/ON-DEMAND-SPARK/smalldata-1000',schema=columns)
df_load.show()
df_load.count()

In [ ]:
sparkSession.stop()

In [ ]:
!/mnt/code/scripts/my_hdfs.sh dfs -ls